<a href="https://colab.research.google.com/github/ruthetum/Deeplearning-NLP/blob/master/machineTranslation(characterLevel).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-Level Neural Machine Translation 구현하기

In [1]:
%tensorflow_version 2.x

import tensorflow as tf

print('version : ', tf.__version__)
print('GPU Available : ', tf.test.is_gpu_available())

version :  2.2.0-rc2
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available :  True


# 1) 데이터 로드

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import pandas as pd
lines = pd.read_csv('gdrive/My Drive/Colab Notebooks/data/fra.txt', sep='\t')
print(len(lines))
lines.head()

175622


,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
0,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,Cours !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,Courez !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Who?,Qui ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


# 2) 정제

In [4]:
lines = lines[["Go.", "Va !"]]
lines.rename(columns={"Go.":"src", "Va !":"tar"},inplace=True)
lines.head()

,src,tar
0,Hi.,Salut !
1,Hi.,Salut.
2,Run!,Cours !
3,Run!,Courez !
4,Who?,Qui ?


In [5]:
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
250,Cool off!,Détends-toi !
49223,It's going to be close.,Ça va être serré.
45653,You're not very funny.,Tu n'es pas très amusant.
28015,Did you drink a lot?,Est-ce que tu as bu beaucoup ?
50293,The reception was warm.,L'accueil était chaleureux.
25657,Stop joking around.,Arrêtez de chahuter !
20960,They kidnapped me.,Elles m'ont enlevée.
25819,The alarm went off.,L'alarme sonna.
16859,We need to leave.,On doit partir.
38290,We got to be friends.,Nous sommes devenus des amis.


In [6]:
# \t : <sos> , \n : <eos> 라고 생각
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
56544,Stay a while and listen.,\t Restez un peu et écoutez ! \n
31958,Tom made a sandwich.,\t Tom prépara un sandwich. \n
7157,Are you lonely?,\t Êtes-vous esseulé ? \n
18017,Do you have a dog?,\t As-tu un chien ? \n
17495,You're all crazy.,\t Vous êtes toutes folles. \n
13595,Close the window.,\t Ferme la fenêtre. \n
22684,Can I see that one?,\t Puis-je voir celui-ci ? \n
45282,Why did they hire you?,\t Pourquoi vous ont-ils recruté ? \n
13687,Do you feel sick?,\t Te sens-tu malade ? \n
817,Keep that.,\t Gardez cela. \n


In [0]:
# 글자 집합 구축
src_vocab=set()

# 1줄씩 읽음
for line in lines.src:
    # 1개의 글자씩 읽음 (프랑스어는 단어가 아니라 글자)
    for char in line:
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [8]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
106


# 3) 정수 인코딩

In [9]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [10]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

**영어 정수 인코딩**

In [11]:
encoder_input = []

#입력 데이터에서 1줄씩 문장을 읽음
for line in lines.src: 
    temp_X = []
    #각 줄에서 1개씩 글자를 읽음
    for w in line:
      # 글자를 해당되는 정수로 변환
      temp_X.append(src_to_index[w])
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2], [46, 57, 64, 23]]


**프랑스어 정수 인코딩**

In [12]:
decoder_input = []

for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2], [1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2], [1, 3, 43, 73, 61, 3, 26, 3, 2]]


**1 이 사라졌다는 것 -> '\t'가 사라졌다는 것**

In [13]:
# 실제 예측값에는 <sos> 여기서는 \t 가 있을 필요가 없음
decoder_target = []

for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2], [3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2], [3, 43, 73, 61, 3, 26, 3, 2]]


In [14]:
# 패딩을 위해 길이 확인
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

25
76


In [0]:
# 영어, 프랑스어 각각 max_len에 맞추어 패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [0]:
# 원-핫 벡터
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# 4) 훈련

In [0]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# 은닉 상태, 셀 상태 -> 컨텍스터 
encoder_states = [state_h, state_c]

In [0]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [19]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
750/750 [==============================] - 18s 24ms/step - loss: 0.7755 - val_loss: 0.6928
Epoch 2/50
750/750 [==============================] - 16s 22ms/step - loss: 0.4803 - val_loss: 0.5537
Epoch 3/50
750/750 [==============================] - 16s 22ms/step - loss: 0.4023 - val_loss: 0.4913
Epoch 4/50
750/750 [==============================] - 16s 22ms/step - loss: 0.3578 - val_loss: 0.4531
Epoch 5/50
750/750 [==============================] - 16s 22ms/step - loss: 0.3283 - val_loss: 0.4299
Epoch 6/50
750/750 [==============================] - 16s 22ms/step - loss: 0.3069 - val_loss: 0.4098
Epoch 7/50
750/750 [==============================] - 16s 22ms/step - loss: 0.2900 - val_loss: 0.3963
Epoch 8/50
750/750 [==============================] - 16s 21ms/step - loss: 0.2765 - val_loss: 0.3875
Epoch 9/50
750/750 [==============================] - 16s 22ms/step - loss: 0.2653 - val_loss: 0.3812
Epoch 10/50
750/750 [==============================] - 16s 22ms/step - loss: 0.255

# 5) seq2seq 기계 번역기 동작

1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻음
2. 상태와 'sos'에 해당하는 '\t'를 디코더로 보냄.
3. 디코더가 'eos'에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복

In [0]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [0]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음
decoder_states = [state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [0]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [0]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [24]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Courez ! 
번역기가 번역한 문장:  Cours ! 
-----------------------------------
입력 문장: I lost.
정답 문장:  J'ai perdu. 
번역기가 번역한 문장:  J'ai perdu. 
-----------------------------------
입력 문장: Come in.
정답 문장:  Entre ! 
번역기가 번역한 문장:  Entre. 
-----------------------------------
입력 문장: I did it.
정답 문장:  Je l'ai fait. 
번역기가 번역한 문장:  Je l'ai fait. 
-----------------------------------
입력 문장: We're shy.
정답 문장:  Nous sommes timides. 
번역기가 번역한 문장:  Nous sommes toujours. 
